    pip install beautifulsoup4
    pip install unidecode

In [1]:
import numpy as np
import unicodedata
import requests
import json
from IPython.core import display
from selenium import webdriver
from bs4 import BeautifulSoup
import os

In [2]:
path = '../data/camptocamp/outiing_list/'
files = os.listdir(path)
files.sort()
files = [f for f in files if not '.npy' in f]
print files

['2006-02-12.txt', '2008-02-14.txt', '2010-03-07.txt', '2012-05-11.txt', '2014-03-29.txt', '2016-03-30.txt', '2018-12-01.txt']


In [3]:
driver = webdriver.Chrome()
driver.implicitly_wait(0.1)

In [4]:
driver.get('https://www.camptocamp.org/outings/1061088/fr/tochuhorn-arete-w-voie-normale')

In [5]:
def extract_data_dict_from_html(html, url):
    text = html.get_text()
    
    # if text.find('Download track') != -1:
    #     print "Found gps"
    #     driver.get(url)
    #     try:
    #         buttons = driver.find_elements_by_class_name('green-btn')
    #         for button in buttons:
    #             button.click()
    #     except:
    #         pass
        
    idx_start = text.find('documentData')
    idx_end = text.find('document.querySelector')
    s = (text[idx_start+15:idx_end]).split(');')[0]
    d = json.loads(s)
    d_add = {
        q: filter(lambda x : len(x), 
                  text[text.find(q):text.find(q)+200].split(':')[1].split('\n'))[0].replace(' ','')
        for q in ['snow_quality', 'avalanche_signs', 'quality'] if text.find(q) > 0
    }
    d.update(**d_add)
    d['has_gps'] = text.find('Download track') != -1
    return d

In [6]:
def get_data_dict(url):
    try:
        r = requests.get(url, timeout=5)
        return extract_data_dict_from_html(BeautifulSoup(r.content, 'html.parser'), url)
    except:
        return False

### Loading all data 

In [7]:
idx_file = -6

In [8]:
file = open(path + files[idx_file])

In [9]:
urls = file.readlines()

In [10]:
data_dict = {}

### Find url with gps 

In [11]:
url_with_gps = []
url = urls[0]

In [12]:
idx = urls.index(url)

In [13]:
idx

0

In [14]:
for url in urls[idx:]:
    display.clear_output(wait=True)
    try:
        print 'loaded', url
        d = get_data_dict(url)
        if d:
        	data_dict[d['document_id']] = d
        else:
        	print 'Failed to load', url
    except:
        print 'Failed to load', url
    

loaded https://www.camptocamp.org/outings/87356/fr/chasseron-traversee-sn-ns



In [15]:
np.save(path + files[idx_file].split('.')[0] + '_data_dict', data_dict)

In [38]:
data_test = np.load(path + files[-1].split('.')[0] + '_data_dict.npy')[()]